In [1]:
import pandas as pd
import quandl
import re
import datetime
from pandas.tseries.offsets import DateOffset
import numpy as np
import itertools
import unicodedata


pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 90)
pd.set_option('display.max_rows', None)
pd.set_option('display.min_rows', 26)
pd.set_option('display.width', 200)

api_key = 'HxxAsWWP1AJofAW8u261'
quandl.ApiConfig.api_key = api_key

In [2]:
"""
US_States = pd.read_html("https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States")
US_States = US_States[1].iloc[:,0:2]
US_States.columns = ["State", "Abbreviation"]

US_States.State = US_States.State.apply(lambda x: x.strip('[D]'))
# a link to a note in wikipedia that is irrelevant to our use-case, so it is removed.

US_States.set_index('State', inplace=True)
"""
# US_States

'\nUS_States = pd.read_html("https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States")\nUS_States = US_States[1].iloc[:,0:2]\nUS_States.columns = ["State", "Abbreviation"]\n\nUS_States.State = US_States.State.apply(lambda x: x.strip(\'[D]\'))\n# a link to a note in wikipedia that is irrelevant to our use-case, so it is removed.\n\nUS_States.set_index(\'State\', inplace=True)\n'

In [3]:
"""
for index, row in US_States.iterrows():
    if index == 'Alabama':
        Temp = quandl.get("FMAC/HPI_" + row.Abbreviation)
        Temp.columns = [index, index]
        SA_Data = pd.DataFrame(Temp.iloc[:,0], index = Temp.index)
        NSA_Data = pd.DataFrame(Temp.iloc[:,1], index = Temp.index)
    else:
        Temp = quandl.get("FMAC/HPI_" + row.Abbreviation)
        Temp.columns = [index, index]
        SA_Data[index] = Temp.iloc[:,0]
        NSA_Data[index] = Temp.iloc[:,1]
"""
# Later we pickle the data so this gathering step is obsolete
# SA_Data

'\nfor index, row in US_States.iterrows():\n    if index == \'Alabama\':\n        Temp = quandl.get("FMAC/HPI_" + row.Abbreviation)\n        Temp.columns = [index, index]\n        SA_Data = pd.DataFrame(Temp.iloc[:,0], index = Temp.index)\n        NSA_Data = pd.DataFrame(Temp.iloc[:,1], index = Temp.index)\n    else:\n        Temp = quandl.get("FMAC/HPI_" + row.Abbreviation)\n        Temp.columns = [index, index]\n        SA_Data[index] = Temp.iloc[:,0]\n        NSA_Data[index] = Temp.iloc[:,1]\n'

In [4]:
# NSA_Data

In [5]:
"""
NSA_Data.to_pickle('Pickles/NSA_Data.pickle')
SA_Data.to_pickle('Pickles/SA_Data.pickle')
US_States.to_pickle('Pickles/US_States.pickle')
"""

"\nNSA_Data.to_pickle('Pickles/NSA_Data.pickle')\nSA_Data.to_pickle('Pickles/SA_Data.pickle')\nUS_States.to_pickle('Pickles/US_States.pickle')\n"

In [6]:
NSA_Data = pd.read_pickle('Pickles/NSA_Data.pickle')
SA_Data = pd.read_pickle('Pickles/SA_Data.pickle')
US_States = pd.read_pickle('Pickles/US_States.pickle')

In [7]:
'''
Hurricanes = pd.read_html("https://en.wikipedia.org/wiki/List_of_Category_5_Atlantic_hurricanes",header=[0],flavor='bs4')
Hurricanes = Hurricanes[1]
# Hurricanes.set_index("Name", inplace = True)
Hurricanes.to_pickle("Pickles/Hurricanes_Wiki.pickle")
'''

'\nHurricanes = pd.read_html("https://en.wikipedia.org/wiki/List_of_Category_5_Atlantic_hurricanes",header=[0],flavor=\'bs4\')\nHurricanes = Hurricanes[1]\n# Hurricanes.set_index("Name", inplace = True)\nHurricanes.to_pickle("Pickles/Hurricanes_Wiki.pickle")\n'

## Changes to be made:
1. Getting both dates into the "yyyy-mm" format, as the HPI data includes only the last day of the month.
1. Getting the Damage information from wikipedia
1. Narrowing down the Areas affected to those in the US and specifically which states involved which hurricane (Manual labour)
1. Assigining an "Overall Strength" feature which may combine different damage reports.

In [8]:
Hurricanes = pd.read_pickle("Pickles/Hurricanes_Wiki.pickle")
Hurricanes

,Name,Dates as aCategory 5,Duration as aCategory 5,Sustainedwind speeds,Pressure,Areas affected,Deaths,Damage(USD),Refs
0,"""Cuba""","October 19, 1924",12 hours,165 mph (270 km/h),910 hPa (26.87 inHg),"Central America, Mexico, CubaFlorida, The Bahamas",90,NaN,[12]
1,"""San Felipe IIOkeechobee""","September 13–14, 1928",12 hours,160 mph (260 km/h),929 hPa (27.43 inHg),"Lesser Antilles, The BahamasUnited States East Coast, Atlantic Canada",4000,$100 million,NaN
2,"""Bahamas""","September 5–6, 1932",1 day,160 mph (260 km/h),921 hPa (27.20 inHg),"The Bahamas, Northeastern United States",16,NaN,NaN
3,"""Camagüey""","November 5–8, 1932",3 days 6 hours,175 mph (280 km/h),915 hPa (27.02 inHg),"Lesser Antilles, Jamaica, Cayman IslandsCuba, The Bahamas, Bermuda",3103,$40 million,NaN
4,"""Cuba–Brownsville""","August 30, 1933",12 hours,160 mph (260 km/h),930 hPa (27.46 inHg),"The Bahamas, Cuba, FloridaTexas, Tamaulipas",179,$27.9 million,NaN
5,"""Tampico""","September 21, 1933",12 hours,160 mph (260 km/h),929 hPa (27.43 inHg),"Jamaica, Yucatán Peninsula",184,$5 million,NaN
6,"""Labor Day""","September 3, 1935",18 hours,185 mph (295 km/h),892 hPa (26.34 inHg),"The Bahamas, Florida, GeorgiaThe Carolinas, Virginia",408,NaN,NaN
7,"""New England""","September 19–20, 1938",18 hours,160 mph (260 km/h),940 hPa (27.76 inHg),"Eastern United States, Southwestern Quebec",682,$306 million,NaN
8,Carol,"September 3, 1953",12 hours,160 mph (260 km/h),929 hPa (27.43 inHg),"Bermuda, New England, Atlantic Canada",5,$2 million,NaN
9,Janet,"September 27–28, 1955",18 hours,175 mph (280 km/h),914 hPa (26.99 inHg),"Lesser Antilles, Central America",1023,$65.8 million,[13]


In [9]:
'''
Hurricanes = pd.read_pickle("Pickles/Hurricanes_Wiki.pickle")
Hurricanes.drop(Hurricanes.tail(1).index,axis=0, inplace=True)
Hurricanes.drop('Refs', axis=1, inplace=True)
Hurricanes.columns = ["Name","During","Duration","Windspeed","Pressure","At","Deaths","Damage"]

Hurricanes.At = Hurricanes.At.apply(lambda x: re.findall('[A-Z][^A-Z]*', x)) 
# write markdown about method (maybe even make tree)

LastPlace = []
for places in Hurricanes.At:
    placeholder = []
    LastPlace.append([])
    for place in places:
        if len(placeholder) == 0:
            placeholder.append(place)
            if (' ' not in place) or (',' in place):
                LastPlace[-1].append(place.strip(', '))
                placeholder = []
            continue
                
        if ' ' == placeholder[-1][-1]:
            placeholder.append(place)
        
        if ',' in placeholder[-1] or ' ' not in placeholder[-1]:
            LastPlace[-1].append(' '.join(placeholder).strip(', '))
            placeholder = []
            
Hurricanes.At = pd.Series(LastPlace)
# The same thing happens for the Hurricane San Felipe II Okeechobee, but... it easier to just correct one data point
Hurricanes.iloc[1,0] = 'San Felipe II Okeechobee'
'''

'\nHurricanes = pd.read_pickle("Pickles/Hurricanes_Wiki.pickle")\nHurricanes.drop(Hurricanes.tail(1).index,axis=0, inplace=True)\nHurricanes.drop(\'Refs\', axis=1, inplace=True)\nHurricanes.columns = ["Name","During","Duration","Windspeed","Pressure","At","Deaths","Damage"]\n\nHurricanes.At = Hurricanes.At.apply(lambda x: re.findall(\'[A-Z][^A-Z]*\', x)) \n# write markdown about method (maybe even make tree)\n\nLastPlace = []\nfor places in Hurricanes.At:\n    placeholder = []\n    LastPlace.append([])\n    for place in places:\n        if len(placeholder) == 0:\n            placeholder.append(place)\n            if (\' \' not in place) or (\',\' in place):\n                LastPlace[-1].append(place.strip(\', \'))\n                placeholder = []\n            continue\n                \n        if \' \' == placeholder[-1][-1]:\n            placeholder.append(place)\n        \n        if \',\' in placeholder[-1] or \' \' not in placeholder[-1]:\n            LastPlace[-1].append(\' \'.

In [10]:
## Damage -> Dollars (float) 
## Pressure -> Pa (float)
## Windspeed -> kmph (float)
## Duration -> Minutes (int)
## During -> dd/mm/yyyy (Date)

In [11]:
'''
Hurricanes.Deaths = Hurricanes.Deaths.apply(int)
Hurricanes.Name = Hurricanes.Name.apply(lambda x: x.strip('" '))
'''

'\nHurricanes.Deaths = Hurricanes.Deaths.apply(int)\nHurricanes.Name = Hurricanes.Name.apply(lambda x: x.strip(\'" \'))\n'

In [12]:
'''
Hurricanes.Damage = Hurricanes.Damage.apply(lambda x: unicodedata.normalize("NFKD", str(x)))

dmg_conversions = dict(million=float(10**6),billion=float(10**9))
dmg_new = []

for damage in Hurricanes.Damage:
    if damage == 'nan':
        dmg_new.append(np.nan)
        continue
        
    cost = damage.split(' ')[0]
    unit = damage.split(' ')[-1]
    
    if unit not in dmg_conversions:
        dmg_new.append(damage)
        print(f'"{damage}" is one of the damage values. It does not match any known units.')
        continue
    
    if '$' not in cost:
        dmg_new.append(damage)
        print(f'This cost: {cost} is not in USD. The cost in dollars will not be calculated.')
        continue
    
    cost = float(cost.strip()[1:])
    dmg_new.append(cost*dmg_conversions[unit])
    
Hurricanes.Damage = pd.Series(dmg_new)
Hurricanes.head(5)
'''

'\nHurricanes.Damage = Hurricanes.Damage.apply(lambda x: unicodedata.normalize("NFKD", str(x)))\n\ndmg_conversions = dict(million=float(10**6),billion=float(10**9))\ndmg_new = []\n\nfor damage in Hurricanes.Damage:\n    if damage == \'nan\':\n        dmg_new.append(np.nan)\n        continue\n        \n    cost = damage.split(\' \')[0]\n    unit = damage.split(\' \')[-1]\n    \n    if unit not in dmg_conversions:\n        dmg_new.append(damage)\n        print(f\'"{damage}" is one of the damage values. It does not match any known units.\')\n        continue\n    \n    if \'$\' not in cost:\n        dmg_new.append(damage)\n        print(f\'This cost: {cost} is not in USD. The cost in dollars will not be calculated.\')\n        continue\n    \n    cost = float(cost.strip()[1:])\n    dmg_new.append(cost*dmg_conversions[unit])\n    \nHurricanes.Damage = pd.Series(dmg_new)\nHurricanes.head(5)\n'

In [13]:
'''
Hurricanes.Pressure = Hurricanes.Pressure.apply(lambda x: unicodedata.normalize("NFKD", str(x)))
Hurricanes.Pressure = Hurricanes.Pressure.apply(lambda x: float(x.split('hPa')[0].strip()))
# only works if hPa is presented first, otherwise use list.index(...)
Hurricanes.head(5)
'''

'\nHurricanes.Pressure = Hurricanes.Pressure.apply(lambda x: unicodedata.normalize("NFKD", str(x)))\nHurricanes.Pressure = Hurricanes.Pressure.apply(lambda x: float(x.split(\'hPa\')[0].strip()))\n# only works if hPa is presented first, otherwise use list.index(...)\nHurricanes.head(5)\n'

In [14]:
'''
Hurricanes.Windspeed = Hurricanes.Windspeed.apply(lambda x: unicodedata.normalize("NFKD", str(x)))
Hurricanes.Windspeed = Hurricanes.Windspeed.apply(lambda x: 
                                                  float(x.split('km/h')[0].strip().split(' ')[-1].strip(' ()')))
Hurricanes.head(5)
'''

'\nHurricanes.Windspeed = Hurricanes.Windspeed.apply(lambda x: unicodedata.normalize("NFKD", str(x)))\nHurricanes.Windspeed = Hurricanes.Windspeed.apply(lambda x: \n                                                  float(x.split(\'km/h\')[0].strip().split(\' \')[-1].strip(\' ()\')))\nHurricanes.head(5)\n'

In [15]:
'''
Hurricanes.Duration = Hurricanes.Duration.apply(lambda x: unicodedata.normalize("NFKD", str(x)))
Min_Conversions = dict(day=1440,days=1440,hour=60,hours=60,minutes=1,minute=1)

# Hurricanes.Duration = Hurricanes.Duration.apply(lambda x: re.findall('[0-9]*[^0-9]*', x)[:-1])

Dura_new = []

for Duration in Hurricanes.Duration:
    minTot = 0
    for timeframe in re.findall('[0-9]+[^0-9]+', Duration):
        number = re.search('[0-9]+',timeframe)
        unit = re.search('[A-Za-z]+',timeframe)
        minTot += int(number.group(0))*Min_Conversions[unit.group(0)]
    Dura_new.append(minTot)

Hurricanes.Duration = pd.Series(Dura_new)
Hurricanes = Hurricanes.rename(columns={"Pressure":"Pressure_hPa", "Damage":"Damage_USD",
                                        "Windspeed":"Windspeed_kmph","Duration":"Duration_Minute"})
Hurricanes.head(5)
'''

'\nHurricanes.Duration = Hurricanes.Duration.apply(lambda x: unicodedata.normalize("NFKD", str(x)))\nMin_Conversions = dict(day=1440,days=1440,hour=60,hours=60,minutes=1,minute=1)\n\n# Hurricanes.Duration = Hurricanes.Duration.apply(lambda x: re.findall(\'[0-9]*[^0-9]*\', x)[:-1])\n\nDura_new = []\n\nfor Duration in Hurricanes.Duration:\n    minTot = 0\n    for timeframe in re.findall(\'[0-9]+[^0-9]+\', Duration):\n        number = re.search(\'[0-9]+\',timeframe)\n        unit = re.search(\'[A-Za-z]+\',timeframe)\n        minTot += int(number.group(0))*Min_Conversions[unit.group(0)]\n    Dura_new.append(minTot)\n\nHurricanes.Duration = pd.Series(Dura_new)\nHurricanes = Hurricanes.rename(columns={"Pressure":"Pressure_hPa", "Damage":"Damage_USD",\n                                        "Windspeed":"Windspeed_kmph","Duration":"Duration_Minute"})\nHurricanes.head(5)\n'

In [16]:
'''
# Problem: can't deal with hurricanes that span multiple months
Hurricanes.During = Hurricanes.During.apply(lambda x: unicodedata.normalize("NFKD", str(x)))
Date_Format = r'([A-Za-z]+)\s?([0-9]*[–]?[0-9]+),\s?([0-9]+)'
# res = re.search(Date_Format,Hurricanes.During[32])
# print(res.group(3))

Date_new = []

for Date in Hurricanes.During:
    res = re.search(Date_Format,Date)
    Year = res.group(3)
    Month = res.group(1)
    Day = res.group(2).split('–')[0]
    # if group(1) is '30' then Day is ['30']'s first element. ('30')
    # and if group(1) is '20-23' then Day is ['20','23']'s first element. ('20')
    Date_new.append(datetime.datetime.strptime(f'{Month} {Day.zfill(2)} {Year}', '%B %d %Y'))
Hurricanes.During = pd.Series(Date_new)
Hurricanes.to_pickle("Pickles/Hurricanes_Edit.pickle")
'''

'\n# Problem: can\'t deal with hurricanes that span multiple months\nHurricanes.During = Hurricanes.During.apply(lambda x: unicodedata.normalize("NFKD", str(x)))\nDate_Format = r\'([A-Za-z]+)\\s?([0-9]*[–]?[0-9]+),\\s?([0-9]+)\'\n# res = re.search(Date_Format,Hurricanes.During[32])\n# print(res.group(3))\n\nDate_new = []\n\nfor Date in Hurricanes.During:\n    res = re.search(Date_Format,Date)\n    Year = res.group(3)\n    Month = res.group(1)\n    Day = res.group(2).split(\'–\')[0]\n    # if group(1) is \'30\' then Day is [\'30\']\'s first element. (\'30\')\n    # and if group(1) is \'20-23\' then Day is [\'20\',\'23\']\'s first element. (\'20\')\n    Date_new.append(datetime.datetime.strptime(f\'{Month} {Day.zfill(2)} {Year}\', \'%B %d %Y\'))\nHurricanes.During = pd.Series(Date_new)\nHurricanes.to_pickle("Pickles/Hurricanes_Edit.pickle")\n'

In [17]:
Hurricanes = pd.read_pickle('Pickles/Hurricanes_Edit.pickle')
Hurricanes.head(5)

,Name,During,Duration_Minute,Windspeed_kmph,Pressure_hPa,At,Deaths,Damage_USD
0,Cuba,1924-10-19,720,270.0,910.0,"[Central America, Mexico, Cuba, Florida, The Bahamas]",90,NaN
1,San Felipe II Okeechobee,1928-09-13,720,260.0,929.0,"[Lesser Antilles, The Bahamas, United States East Coast, Atlantic Canada]",4000,100000000.0
2,Bahamas,1932-09-05,1440,260.0,921.0,"[The Bahamas, Northeastern United States]",16,NaN
3,Camagüey,1932-11-05,4680,280.0,915.0,"[Lesser Antilles, Jamaica, Cayman Islands, Cuba, The Bahamas, Bermuda]",3103,40000000.0
4,Cuba–Brownsville,1933-08-30,720,260.0,930.0,"[The Bahamas, Cuba, Florida, Texas, Tamaulipas]",179,27900000.0


In [18]:
from IPython.display import display

display(NSA_Data)

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,elaware,Florida,Georgia,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1975-01-31,35.167764,34.604936,28.967806,37.770761,15.812954,19.732791,24.125223,27.746994,30.747651,30.126544,27.206659,32.487773,25.765938,32.187754,33.243103,33.418666,31.655667,33.134993,27.945117,26.955122,15.922510,24.870520,25.467077,43.039507,30.634195,27.383052,31.105418,26.842140,24.565586,22.612114,28.191536,23.839583,27.864700,40.615577,30.110158,38.461932,19.352480,29.061182,20.571629,31.299703,35.825137,31.938069,32.818733,24.739254,26.571923,27.628050,17.511135,42.838255,27.783014,31.174526
1975-02-28,35.529189,35.035870,29.450035,37.803814,15.890206,19.923528,24.681892,27.916509,32.025110,29.874077,27.188735,32.604908,25.688449,32.794171,33.773369,33.744510,31.762795,33.590706,27.864975,26.920690,15.829138,25.111263,25.719125,42.402598,32.072269,27.552216,31.057567,27.384421,24.631964,22.214310,28.633318,23.314171,28.355607,40.516202,30.185510,38.277973,19.924206,29.362159,20.932120,31.506294,35.617404,32.033773,32.942274,25.074462,26.831612,27.936771,17.643281,43.984922,28.162877,31.669633
1975-03-31,35.854669,35.407964,29.848097,37.826190,16.048152,20.107532,25.138555,27.967729,33.911579,29.747488,27.226005,32.618621,25.757826,33.266373,34.161074,34.078449,31.815572,34.031234,27.755241,26.835583,15.728765,25.187753,25.909498,41.809097,33.479234,27.677570,31.016547,27.906726,24.751778,21.982049,29.027771,23.128211,28.743709,40.388627,30.273529,38.076446,20.438580,29.582119,21.298219,31.622041,35.365400,32.047196,33.298538,25.311783,27.056628,28.141810,17.748555,45.135358,28.442934,32.156860
1975-04-30,36.204131,35.784216,30.103491,37.854430,16.316849,20.293914,25.414024,28.055369,36.184107,29.763036,27.286179,32.655965,25.817514,33.619970,34.414152,34.440082,31.898090,34.277904,27.687999,26.718133,15.582040,25.145926,26.065720,41.391771,34.594752,27.747390,30.921804,28.368064,24.778276,21.940950,29.278960,23.135118,28.948549,40.327166,30.458394,37.868604,20.891833,29.698539,21.686043,31.649013,35.120367,32.153273,34.029014,25.465186,27.247946,28.225428,17.849691,46.337792,28.635794,32.602327
1975-05-31,36.487839,36.228856,30.168881,37.899527,16.546772,20.407527,25.522871,28.135907,36.339774,29.816791,27.350164,32.599072,25.851692,33.738726,34.475539,34.773180,32.060269,34.320450,27.735038,26.749860,15.486004,25.012782,26.217121,41.120603,34.456609,27.769800,30.907366,28.773535,24.731807,22.054274,29.427212,23.194721,29.115238,40.421598,30.663461,37.675066,21.293034,29.816999,22.046790,31.608954,34.917177,32.258502,34.168357,25.504588,27.467311,28.325018,17.901681,47.534735,28.747946,32.968711
1975-06-30,36.594636,36.720667,30.002919,38.032395,16.616038,20.489638,25.505477,28.246312,34.526798,29.807098,27.428591,32.498133,25.884581,33.661582,34.474612,35.021292,32.356302,34.340218,27.896109,26.869061,15.429134,24.845398,26.376509,40.975019,33.270751,27.809091,30.921803,29.114152,24.566302,22.196820,29.557842,23.321597,29.167956,40.638624,30.888571,37.612123,21.661281,30.033000,22.314141,31.586146,34.793162,32.393513,33.653732,25.458098,27.714906,28.429245,17.977443,48.635003,28.816322,33.295337
1975-07-31,36.482482,37.284277,29.651366,38.116182,16.671671,20.549340,25.411680,28.327649,33.288860,29.713681,27.515095,32.444139,26.026435,33.475332,34.522879,35.181618,32.792763,34.545651,28.171572,27.089304,15.455696,24.820955,26.651842,40.859549,32.226956,27.875355,30.924594,29.359546,24.335795,22.369428,29.691938,23.536048,29.145711,41.008682,31.089762,37.732866,21.998376,30.273738,22.432213,31.673

In [19]:
Hurricanes = pd.read_pickle('Pickles/Hurricanes_Edit.pickle')
NSA_Data.loc[Hurricanes.During[20]:Hurricanes.During[21]]

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,elaware,Florida,Georgia,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1992-08-31,73.707069,74.204338,66.340115,74.098075,76.362018,50.289835,81.855837,81.409207,73.166312,66.569439,113.764640,72.329989,70.490026,72.350814,67.338820,67.180783,70.453372,68.478473,77.278658,83.655350,61.960637,60.612327,60.035951,71.895060,69.454377,64.828159,68.502068,80.560721,65.550469,75.751941,74.178384,77.779782,70.165902,74.129976,71.526098,71.631002,60.827896,84.263904,80.769164,71.263663,68.562418,70.105390,70.804709,56.804008,81.722378,77.598362,68.938017,76.286074,65.057986,65.113215
1992-09-30,74.203438,74.379731,66.489542,74.366537,76.155993,50.653674,82.072174,81.671659,73.368495,66.813262,113.861164,72.852383,70.830323,72.699576,67.575145,67.306631,71.002375,68.915439,77.352332,84.204968,61.964078,61.007306,60.257946,72.074868,69.713010,65.493378,68.868945,80.722057,65.628227,76.078757,74.529061,78.009386,70.327794,74.519750,71.891941,71.945503,61.460923,84.356643,80.920565,71.459358,69.247345,70.306654,71.133838,57.461405,81.728391,77.829639,69.475079,76.775809,65.372843,65.721375
1992-10-31,74.442516,74.679562,66.505756,74.625575,75.793479,51.083691,82.208265,81.918130,73.550579,67.003250,114.069839,73.506795,71.190973,72.992345,67.864815,67.588591,71.396701,69.365469,77.396377,84.224773,61.943957,61.372891,60.506795,72.180057,69.912626,66.214751,69.298408,80.777192,65.556594,76.281220,74.802356,78.358422,70.389027,74.887935,72.203020,72.292332,62.081647,84.490730,80.910554,71.554150,69.816806,70.427353,71.451235,58.102632,81.616104,77.960160,70.150317,77.389267,65.771656,66.282315
1992-11-30,74.608454,75.057086,66.590458,74.873553,75.544229,51.524413,82.107291,82.297170,73.721784,66.909091,114.363053,74.210685,71.489040,73.265778,68.149912,67.856349,71.685964,69.681586,77.075768,83.974051,62.033301,61.625408,60.745328,72.220008,70.150493,67.011880,69.660424,80.834198,65.351103,76.390167,75.093389,78.648142,70.560137,75.233039,72.479115,72.604427,62.735915,84.622677,80.609021,71.573228,70.167090,70.544940,71.597236,58.729980,81.542902,78.078796,70.518641,78.039054,66.181152,66.663945
1992-12-31,74.748563,75.312412,66.672385,75.097471,75.036708,51.871761,81.652606,82.504299,73.537038,66.749576,114.452051,74.907784,71.664037,73.603206,68.460151,68.036885,71.790885,69.725725,76.427096,83.676177,61.754708,61.857335,60.880609,72.181159,70.342616,67.852682,69.788182,80.868221,64.925440,76.329653,75.412803,78.478255,70.813273,75.555647,72.625204,72.752275,63.206559,84.807164,80.231977,71.539724,70.381957,70.672302,71.629065,59.279875,81.582336,77.865514,70.429958,78.557272,66.643055,66.843795
1993-01-31,74.907770,75.406435,66.797933,75.206703,74.256739,52.229734,80.877997,82.376077,73.299175,66.720283,114.704648,75.143265,71.813778,73.757727,68.781633,68.230355,71.783008,69.923009,75.728508,83.298967,60.989128,61.912931,60.988217,72.241201,70.552949,68.570869,70.016742,80.829289,64.417763,76.085318,75.672007,78.065302,70.773008,75.868479,72.675568,72.892325,63.364494,84.804240,79.921015,71.479322,70.612632,70.748691,71.394829,59.722331,81.656269,77.630043,70.496312,78.935205,67.049287,67.018732
1993-02-28,75.164631,75.379277,66.746456,75.369898,73.702813,52.525570,80.120730,82.115966,73.198061,66.867964,114.878045,75.252916,71.943657,73.967293,69.062137,68.476061,71.873493,70.086458,75.262849,83.069983,60.429893,62.718983,61.179819,72.440889,70.471681,69.126560,70.351746,80.739672,63.906106,75.852779,75.972813,77.712371,70.791009,76.270674,72.890170,73.071086,63.622970,84.748030,79.712696